# Import Enzyme Aligned Datafile

In [1]:
import helper

In [2]:
enzyme_aligned_datafile = 'Data/Modified-Enzyme-Alignment.txt'

X,y,enz_names = helper.parseEnzymeFile(enzyme_aligned_datafile)

# Perform a modified split by setting a random seed

In [3]:
import numpy as np
np.random.seed(350)
te_idx,tr_idx = helper.modified_split(enz_names)

# Look separately at positions-motifs

In [4]:
from IndModels.models import NGModel,GAACModel

In [5]:
total_length = len(X[0])

In [6]:
X = np.array(list(map(list,X)))

In [7]:
X[:,[i for i in range(25,125)]].shape

(116, 100)

In [ ]:
aligned - 533 
1 enzyme multiple 
first pred 0 to 100 ; 25 to 125 ....

In [8]:
band_length = 100
motif_length = 7
band_step = 25
motif_step = 1

band_number = 0
curr_extreme = band_number*band_step + band_length

test_preds,test_acc,train_acc = dict(),dict(),dict()
test_preds_ng,test_acc_ng,train_acc_ng = dict(),dict(),dict()

while curr_extreme<total_length:
    X_sliced = X[:,[i for i in range(band_number*band_step,band_number*band_step+band_length)]]
    posModel = GAACModel(enz_names,X_sliced,y,tr_idx,te_idx,k=motif_length,s=motif_step,random_seed=350)
    posModel2 = NGModel(enz_names,X_sliced,y,tr_idx,te_idx,k=motif_length,s=motif_step,random_seed=350)
    test_acc[band_number] = posModel.SVMobject.acc_test
    train_acc[band_number] = posModel.SVMobject.acc_train
    test_preds[band_number] = posModel.SVMobject.ypredtest
    
    test_acc_ng[band_number] = posModel2.SVMobject.acc_test
    train_acc_ng[band_number] = posModel2.SVMobject.acc_train
    test_preds_ng[band_number] = posModel2.SVMobject.ypredtest

    band_number+=1
    curr_extreme = band_number*band_step + band_length

In [9]:
y[te_idx]

array([2, 2, 3, 2, 3, 2, 3, 2, 2, 2, 1, 3, 3, 3, 1, 1, 3, 3, 3, 3, 1, 3,
       1, 3, 1, 3, 2])

In [10]:
sum(np.array(list(train_acc.values()))/0.93)

15.313311086392286

In [11]:
def get_weighted_bands(train_acc):
    train_acc_values = np.array(list(train_acc.values()))
    divide_by = max(train_acc_values)
    new_acc_values = train_acc_values/divide_by
    sum_new_vals = sum(new_acc_values)
    weighted_vals = new_acc_values/sum_new_vals
    weigh_bands = dict(zip(train_acc.keys(),weighted_vals))
    return weigh_bands
    

In [1]:
6*25

150

In [3]:
12*25+ 100

400

In [12]:
get_weighted_bands(train_acc)

{0: 0.04277643260694108,
 1: 0.04681194511702986,
 2: 0.05004035512510089,
 3: 0.05326876513317191,
 4: 0.05326876513317191,
 5: 0.05326876513317191,
 6: 0.062146892655367235,
 7: 0.06133979015334948,
 8: 0.06537530266343825,
 9: 0.0645682001614205,
 10: 0.06295399515738499,
 11: 0.059725585149313955,
 12: 0.05649717514124294,
 13: 0.05488297013720742,
 14: 0.05326876513317191,
 15: 0.05326876513317191,
 16: 0.05326876513317191,
 17: 0.05326876513317191}

In [13]:
weighted_bands = get_weighted_bands(train_acc)
weighted_bands_ng = get_weighted_bands(train_acc_ng)

In [14]:
def get_weighted_preds(band_preds):
    labels = {1:0,2:0,3:0}
    for b,yhat in enumerate(band_preds):
        labels[yhat] += weighted_bands[b]
    return labels

In [15]:
def get_band_preds(idx):
    return [v[idx] for v in test_preds.values()]

In [16]:
def get_weighted_preds_ng(band_preds_ng):
    labels = {1:0,2:0,3:0}
    for b,yhat in enumerate(band_preds_ng):
        labels[yhat] += weighted_bands_ng[b]
    return labels

def get_band_preds_ng(idx):
    return [v[idx] for v in test_preds_ng.values()]

In [17]:
weighted_bands

{0: 0.04277643260694108,
 1: 0.04681194511702986,
 2: 0.05004035512510089,
 3: 0.05326876513317191,
 4: 0.05326876513317191,
 5: 0.05326876513317191,
 6: 0.062146892655367235,
 7: 0.06133979015334948,
 8: 0.06537530266343825,
 9: 0.0645682001614205,
 10: 0.06295399515738499,
 11: 0.059725585149313955,
 12: 0.05649717514124294,
 13: 0.05488297013720742,
 14: 0.05326876513317191,
 15: 0.05326876513317191,
 16: 0.05326876513317191,
 17: 0.05326876513317191}

In [ ]:
150-250; 300-400

In [18]:
get_band_preds(2),get_band_preds_ng(2)

([3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 1, 1, 3, 3, 3, 3, 3, 3],
 [3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 1, 1, 3, 3, 3, 3, 3, 3])

In [19]:
get_weighted_preds(get_band_preds(7)),get_weighted_preds_ng(get_band_preds_ng(7))

({1: 0.08958837772397094, 2: 0.24939467312348665, 3: 0.6610169491525423},
 {1: 0.04347826086956522, 2: 0.3735909822866345, 3: 0.5829307568438005})

In [20]:
y[te_idx]

array([2, 2, 3, 2, 3, 2, 3, 2, 2, 2, 1, 3, 3, 3, 1, 1, 3, 3, 3, 3, 1, 3,
       1, 3, 1, 3, 2])

In [21]:
def get_pred_and_actual(idx):
    return get_weighted_preds(get_band_preds(idx)),y[te_idx][idx]

def get_pred_and_actual_ng(idx):
    return get_weighted_preds_ng(get_band_preds_ng(idx)),y[te_idx][idx]

In [22]:
get_pred_and_actual(26),get_pred_and_actual_ng(26)

(({1: 0.9467312348668281, 2: 0, 3: 0.05326876513317191}, 2),
 ({1: 1.0000000000000002, 2: 0, 3: 0}, 2))

In [26]:
get_band_preds(5),get_band_preds_ng(5)

([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [24]:
for i in range(len(te_idx)):
    print(i,enz_names[te_idx][i])
    print(get_pred_and_actual(i))
    print(get_pred_and_actual_ng(i))

0 Cuphea_viscosisssima_(CvB2MT28
({1: 0.3131557707828894, 2: 0, 3: 0.6868442292171105}, 2)
({1: 0.31078904991948475, 2: 0.06280193236714977, 3: 0.6264090177133657}, 2)
1 Cuphea_viscosisssima_(CvB2MT21
({1: 0.3131557707828894, 2: 0, 3: 0.6868442292171105}, 2)
({1: 0.24637681159420294, 2: 0, 3: 0.7536231884057972}, 2)
2 Cuphea_viscosisssima_(CvB2MT32
({1: 0.12267958030669895, 2: 0.12994350282485873, 3: 0.7473769168684422}, 3)
({1: 0.12238325281803544, 2: 0.13043478260869568, 3: 0.747181964573269}, 3)
3 Cocos_nucifera_(CnFatB3)
({1: 0.8401937046004843, 2: 0, 3: 0.1598062953995157}, 2)
({1: 0.9484702093397748, 2: 0, 3: 0.05152979066022545}, 2)
4 Cuphea_viscosisssima_(CvB2MT26
({1: 0, 2: 0, 3: 1.0}, 3)
({1: 0, 2: 0, 3: 1.0000000000000002}, 3)
5 Cuphea_carthagenesis_(CcFatB)
({1: 0, 2: 0, 3: 1.0}, 2)
({1: 0.053140096618357495, 2: 0, 3: 0.9468599033816427}, 2)
6 Cuphea_aequipetala_(Ca1FatB2)
({1: 0, 2: 0, 3: 1.0}, 3)
({1: 0, 2: 0, 3: 1.0000000000000002}, 3)
7 rTE40
({1: 0.08958837772397094, 2